In [ ]:
import numpy as np
import tqdm
import operator
import lightgbm as lgb
import os

In [ ]:
#data 
queries = {}
with open('norm_queries.tsv', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip('\n').split('\t')
        if line[0] == '':
            line.pop(0)
        queries[line[0]] = line[1]
f.close()
        
titles= {}
with open("norm_titles.tsv", encoding='utf-8') as f:
    for line in f.readlines():
        line=line.split('\t')
        titles[line[0]]= line[1][:-1]
f.close()
        
urls = {}
with open('url.data', encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    line = line[:-1].split('\t')
    if line[1][:7] == 'http://':
        line[1] = line[1][7:]
    if line[1][:4] == 'www.':
        line[1] = line[1][4:]
    urls[line[0]] = line[1].split('/')[0]
f.close()

In [ ]:

train=[]
with open("train.marks.tsv", encoding='utf-8') as fin:
    for line in fin.readlines():
        line=line[:-1].split('\t')
        train.append([line[0],line[1], line[2]])


                
test=[]
with open("sample.csv", encoding='utf-8') as fin:
    fin.readline()
    for line in fin.readlines():
        line=line[:-1].split(',')
        test.append([line[0],line[1],-1])

In [ ]:
for q in all_queries:
    if q + '.txt' not in os.listdir(out_dir):
        tfidf_score(q, (1,3), 'text_tfids_13')

TfIdf scores с ngram_range=(1,3) для текстов и запросов

In [ ]:
def get_text_feature(input_dir):
    feature_dict = {}
    files = os.listdir(input_dir)
    for file in files:
        with open(input_dir + '/' + file, encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            line = line[:-1].split('\t')
            feature_dict[(line[0], line[1])] = np.float64(line[2])
    return feature_dict

In [ ]:
text_tfids_13 = get_text_feature('text_tfids_13')

TfIdf scores с ngram_range=(3,7) для текстов и запросов text_tfids_37

In [ ]:
text_tfids_37 = get_text_feature('text_tfids_37')

TfIdf scores с ngram_range=(1,3) для заголовков и запросов title_tfids_13

In [ ]:
title_tfids_13 = get_text_feature('title_tfids_13')

TfIdf scores с ngram_range=(3,8) для заголовков и запросов title_tfids_38

In [ ]:
title_tfids_38 = get_text_feature('title_tfids_38')

BM25Okapi score для текстов и запросов bm_texts

In [ ]:
bm_texts = get_text_feature('bm_texts')

passage score для текстов и запросов, window=3, stride=1 pass_text_31

In [ ]:
pass_text_31 = get_text_feature('pass_text_31')

passage score для текстов и запросов, window=3, stride=2 pass_text_32

In [ ]:
pass_text_32 = get_text_feature('pass_text_32')

passage score для текстов и запросов, window=7, stride=3 pass_text_73

In [ ]:
pass_text_73 = get_text_feature('pass_text_73')

passage score для заголовков и запросов, window=3, stride=1 pass_title_31

In [ ]:
pass_title_31 = get_text_feature('pass_title_31')

use score для заголовков и запросов

In [ ]:
use = get_text_feature('use')

In [ ]:
quse = get_text_feature('quse')

d2v score для заголовков и запросов

In [ ]:
d2v = get_text_feature('d2v')

fasttext score для заголовков и запросов

In [ ]:
fasttext = get_text_feature('fasttext')

Списки похожих запросов

In [ ]:
similar_queries = {}
with open('similar_queries.txt', encoding='utf-8') as fin:
    lines = fin.readlines()
    for line in lines:
        spl = line[:-2].split('\t')
        if len(spl) == 1:
            continue
        tmp = spl[1].split(' ')
        for i in range(13):
            tmp.pop()
        similar_queries[spl[0]] = tmp

Поведенческие признаки

In [ ]:
file =open("sDBN/part-r-00000" ,'r', encoding='utf-8')
doc2sdbn = {}
for line in file.readlines():
    l=line[:-1].split('\t')
    k = l[0]
    doc2sdbn[k]= np.array(l[1:], dtype = np.float64)
file.close()

In [ ]:
urlfiles = os.listdir('url_q_features')
uf = {}
quf = {}
for file in urlfiles:
    with open('url_q_features/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        line.pop()
        if file[1] == 'q':
            q = line.pop(0)
            u = line.pop(0)
            quf[(q,u)] = np.array(line, dtype = np.float64)
        else:
            u = line.pop(0)
            uf[u] = np.array(line, dtype = np.float64)

In [ ]:

file =open("q_features/part-r-00000" ,'r', encoding='utf-8')
qf = {}
for line in file.readlines():
    l=line[:-1].split('\t')
    k = l[0]
    l.pop(0)
    qf[k]= np.array(l, dtype = np.float64)
file.close()

In [ ]:
def create_dataset(cat):
    global similar_queries
    x = []
    y = []
    qid = []
    index = []
    for entry in tqdm.tqdm(cat):
        try:
            # sDBN
            try:
                tempx = np.hstack((tempx,doc2sdbn[entry[1]]))
            except:
                temp = None
                if temp is None:
                    [temp] = [[0.0] * 3]
                tempx = np.hstack((tempx,temp))
            # query features
            try:
                tempx = np.hstack((tempx,qf[entry[0]]))
            except:
                temp = None
                if temp is None:
                    for q in similar_queries[entry[0]]:
                        if q in qf.keys():
                            temp = qf[q]
                            break
                if temp is None:
                    [temp] = [[0.0] * 23]
                tempx = np.hstack((tempx,temp))
            # url features
            try:
                tempx = np.hstack((tempx,uf[entry[1]]))
            except:
                temp = None
                
                for q in similar_queries[entry[0]]:
                    if entry[1] in uf.keys():
                        temp = uf[entry[1]]
                        break
                if temp is None:
                    [temp] = [[0.0] * 13]
                tempx = np.hstack((tempx,temp))
            # query-url features
            try:
                tempx = np.hstack((tempx,quf[(entry[0],entry[1])]))
            except:
                temp = None

                for q in similar_queries[entry[0]]:
                    if (q,entry[1]) in quf.keys():
                        temp = quf[(q,entry[1])]
                        break
                if temp is None:
                    [temp] = [[0.0] * 43]
                tempx = np.hstack((tempx,temp))

            try:
                tempx = np.hstack((tempx,text_tfids_13[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,text_tfids_37[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,title_tfids_13[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,title_tfids_38[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,bm_texts[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,pass_text_31[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,pass_text_32[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,pass_text_73[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,pass_title_31[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,use[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,quse[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,d2v[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,fasttext[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))

            x.append(tempx)
            y.append(float(entry[2]))
            qid.append(int(entry[0]))
            index.append((entry[0], entry[1]))
        except:
            continue
    X = np.row_stack([row for row in x])
    Y = np.row_stack([row for row in y])
    return X, Y, qid, index

In [ ]:
X_train, Y_train, qid_train, _ = create_dataset(train)

In [ ]:
X_test, Y_test, _ , index = create_dataset(test)

In [ ]:
qids_unique = np.unique(qid_train, return_counts=True)[1]
lgbm_ranker = lgb.LGBMRanker(objective='lambdarank', max_depth=12, n_estimators= 1000, subsample=0.8, learning_rate=0.01, num_leaves=64)

In [ ]:
%%time
lgbm_ranker.fit(X_train,Y_train.ravel(),group=qids_unique)

In [ ]:
%%time
y_pred = lgbm_ranker.predict(X_test)

In [ ]:
lgbm_ranker.feature_importances_

In [ ]:
def create_submission(y_pred):
    query_doc = {}
    query_doc_score = {}
    
    with open('sample.csv', 'r', encoding='UTF-8') as f:
        sample = f.readlines()

        for i in range(1,len(sample)):

            l = sample[i][:-1].split(',')
            if l[0] not in query_doc.keys():
                query_doc[l[0]] = []
            else:
                query_doc[l[0]].append(l[1])
            query_doc_score[tuple(l)] = y_pred[i-1]
    results = []
    for query in query_doc:
        res = {}
        for doc in query_doc[query]:
            try:
                res[doc] = query_doc_score[(query,doc)]
            except:
                continue
        res_sorted = sorted(res.items(), key = operator.itemgetter(1), reverse = True)
        for i in range(len(res_sorted)):
            results.append([query,res_sorted[i][0]])
    return results

In [ ]:
results = create_submission(y_pred)

In [ ]:
with open('submit_tr.csv', 'w') as fout:
    fout.write('QueryId,DocumentId\n')
    for result in results:
        fout.write(result[0] + ',' + result[1]+'\n')